<a href="https://colab.research.google.com/github/ShutTheFuckUpDonny/AI_course/blob/master/%D0%BD%D0%B5%D0%B4%D0%B5%D0%BB%D1%8F_12_%D0%9F%D0%BE%D0%BB%D0%BD%D0%BE%D1%81%D0%B2%D1%8F%D0%B7%D0%BD%D1%8B%D0%B5_%D1%81%D0%B5%D1%82%D0%B8%2C_%D0%BE%D0%B1%D1%83%D1%87%D0%B0%D1%8E%D1%89%D0%B0%D1%8F_%D0%B8_%D1%82%D0%B5%D1%81%D1%82%D0%BE%D0%B2%D0%B0%D1%8F_%D0%B2%D1%8B%D0%B1%D0%BE%D1%80%D0%BA%D0%B8_Pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Повысьте точность модели по обнаружению мин до 90 % на тестовой выборке. 

Можно использовать различные варианты слоев Dropout и BatchNormalization. Можно менять количество примеров в обучающей и проверочной выборках, но нельзя менять количество примеров в тестовой.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import utils
from tensorflow.keras.optimizers import Adam, Adadelta
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
import pandas as pd
from google.colab import files

In [ ]:
files.upload()
!ls

Saving sonar.csv to sonar.csv
sample_data  sonar.csv


In [ ]:
df = pd.read_csv('sonar.csv', header=None)
print(df.head())
print(df.shape)

       0       1       2       3       4   ...      56      57      58      59  60
0  0.0200  0.0371  0.0428  0.0207  0.0954  ...  0.0180  0.0084  0.0090  0.0032   R
1  0.0453  0.0523  0.0843  0.0689  0.1183  ...  0.0140  0.0049  0.0052  0.0044   R
2  0.0262  0.0582  0.1099  0.1083  0.0974  ...  0.0316  0.0164  0.0095  0.0078   R
3  0.0100  0.0171  0.0623  0.0205  0.0205  ...  0.0050  0.0044  0.0040  0.0117   R
4  0.0762  0.0666  0.0481  0.0394  0.0590  ...  0.0072  0.0048  0.0107  0.0094   R

[5 rows x 61 columns]
(208, 61)


In [ ]:
dataset = df.values                     # Берем только значения массива(без индексов)
X = dataset[:,0:60].astype('float')     # Присваиваем им тип данных - float(с плавающей точкой) данным с 0 по 60 колонки
Y = dataset[:,60]                       # Присваеваем значению Y данные из столбца с индексом 60
Y[Y=='R'] = '0'                         # Если значение элемента столбца равно 'R', присваеваем ему значение '0'
Y[Y=='M'] = '1'                         # Если значение элемента столбца равно 'Y', присваеваем ему значение '1'
Y = Y.astype(int)                       # Меняем тип данных столбца на 'int'(целочисленный тип данных)
print(X.shape)
print(Y.shape)
print(Y)


(208, 60)
(208,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape) 

(166, 60)
(42, 60)
(166,)
(42,)


In [ ]:
def create_model():
    model = Sequential()
    model.add(Dense(60, input_dim=60, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(30, activation='relu'))
    #model.add(Dropout(0.3))
    #model.add(Dense(20, activation='relu'))
    #model.add(Dropout(0.2))
    #model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
    return model

In [ ]:
def fit_model():
    model = create_model()
    model.fit(x_train, y_train, batch_size=4, epochs=150, validation_split=0.2, verbose=1)
    scores = model.evaluate(x_test, y_test, verbose=1)
    print(scores)
    print("Доля верных ответов на тестовых данных, в процентах: ", round(scores[1] * 100, 4), "%", sep="")

In [ ]:
fit_model()

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: 0.7005 - accuracy: 0.4697 - val_loss: 0.6842 - val_accuracy: 0.6765
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.6870 - accuracy: 0.5227 - val_loss: 0.6760 - val_accuracy: 0.6471
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.6733 - accuracy: 0.6136 - val_loss: 0.6675 - val_accuracy: 0.5882
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.6604 - accuracy: 0.6515 - val_loss: 0.6454 - val_accuracy: 0.6765
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.6707 - accuracy: 0.5833 - val_loss: 0.6299 - val_accuracy: 0.7059
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.6417 - accuracy: 0.6439 - val_loss: 0.6113 - val_accuracy: 0.7059
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.6262 - accuracy: 0.6742 - val_loss: 0.5882 - val_accuracy: 0.7647
Epoch 8/150
3

BatchNormalization ухудшает точность.
При текущей модели точность достигается 85%.
Dropout помогает бороться с переобучаемостью и точность на тестовых данных занчительно выше. Достигнуть 90% точности не удается, возможно, по причине небольшого размера общей выборке